# Contexto de Negocio y Marco de Trabajo

## Empresa y situación
El flujo de pedidos crece y requiere actualizaciones frecuentes del lakehouse. Reprocesar todo a diario es costoso.

## Qué / Por qué / Para qué / Cuándo / Cómo
- Qué: Pipeline incremental de órdenes en Parquet.
- Por qué: Reducir tiempo y costos de cómputo vs recargas completas.
- Para qué: Mantener datos frescos para analítica y monitoreo operativo.
- Cuándo: Cada 1–4 horas según criticidad.
- Cómo: Checkpoint de fecha, filtrado incremental, append y actualización de checkpoint.

In [ ]:
# Configuración del entorno
import sys
from pathlib import Path

# Agregar el directorio raíz al path para imports
root_dir = Path.cwd().parent.parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

print(f"✅ Root directory agregado al path: {root_dir}")

# Contexto de Negocio y Marco de Trabajo

## Empresa y situación
Retailer con sistema de órdenes distribuido que genera archivos diarios. Se necesita un pipeline incremental eficiente para procesar solo las órdenes nuevas o modificadas.

## Qué / Por qué / Para qué / Cuándo / Cómo
- Qué: Pipeline incremental que detecta y procesa solo las órdenes nuevas desde la última ejecución.
- Por qué: Evitar reprocesar todo el histórico diariamente ahorra tiempo y recursos computacionales.
- Para qué: Mantener un data warehouse actualizado con baja latencia para análisis near real-time.
- Cuándo: Ejecutar cada hora o cada 4 horas según la criticidad del negocio.
- Cómo: Usar checkpoints para rastrear la última fecha procesada y filtrar incrementalmente.

---
id: "DE-02"
title: "Pipeline incremental de órdenes"
specialty: "Data Engineering"
process: "Deliver"
level: "Intermediate"
tags: ["etl", "incremental", "orders", "python", "parquet"]
estimated_time_min: 45
---

import pandas as pd
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

ensure_dirs()
logger = get_logger('DE-02')
logger.info('Iniciando pipeline incremental de órdenes')
print('✅ Librerías cargadas y rutas preparadas')
---

In [1]:
# ⚙️ Preparación de entorno y rutas
# Si esta celda tarda demasiado o se cuelga:
# 1) Abre la paleta de comandos (Ctrl+Shift+P)
# 2) "Jupyter: Restart Kernel"
# 3) "Run All Above/Below" o ejecuta desde la primera celda

import sys
from pathlib import Path

# Detectar raíz del repo (buscando pyproject.toml o carpeta src)
_candidates = [Path.cwd(), *Path.cwd().parents]
_repo_root = None
for _p in _candidates:
    if (_p / 'pyproject.toml').exists() or (_p / 'src').exists():
        _repo_root = _p
        break
if _repo_root is None:
    _repo_root = Path.cwd()

if str(_repo_root) not in sys.path:
    sys.path.insert(0, str(_repo_root))

print(f"✅ Entorno listo. Raíz del repo: {_repo_root}")

✅ Entorno listo. Raíz del repo: f:\GitHub\supply-chain-data-notebooks


orders = pd.read_csv(DATA_RAW / 'orders.csv')
orders['date'] = pd.to_datetime(orders['date'])
# Filtrado incremental robusto usando Timestamp
new_orders = orders[orders['date'] > last_ts]
logger.info(f'Nuevas órdenes: {len(new_orders)}')
print(new_orders.head())
Procesar toda la historia diariamente es costoso. Un pipeline incremental reduce tiempo y recursos.

### ¿Para qué?
- Mantener actualizado el lakehouse sin recargas completas
- Habilitar near real-time analytics
- Optimizar uso de infraestructura

### ¿Cuándo?
Ejecutar cada hora o cada 4 horas según criticidad del negocio.

### ¿Cómo?
1. Leer última fecha procesada desde checkpoint
2. Filtrar órdenes con `date > last_processed`
3. Append a tabla existente
4. Actualizar checkpoint

In [ ]:
import pandas as pd
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

ensure_dirs()
logger = get_logger('DE-02')
logger.info('Iniciando pipeline incremental de órdenes')
print('✅ Librerías cargadas y rutas preparadas')